In [84]:
from fastai.tabular.all import load_pickle, patch
from src.constants import DATA_PATH, USER_PATH
from src.data import gedi_raster_matching
from src.processing.rf import split_data
from src.data import shape_processor
import geopandas as gpd
from fastai.tabular.all import *
from sklearn.metrics import *
from src.processing.rf import train
import importlib
importlib.reload(train)

2023-06-16 08:49:07,199 DEBUG: Logger /home/jk871/fire-regen/src/processing/rf/train.py already set up. [in get_logger at /home/jk871/fire-regen/src/utils/logging_util.py:51]


<module 'src.processing.rf.train' from '/home/jk871/fire-regen/src/processing/rf/train.py'>

In [85]:
sierras = gpd.read_file(f"{USER_PATH}/data/shapefiles/sierras.shp")
sierras_box = shape_processor.get_box(sierras)

In [86]:
train.logger.setLevel("DEBUG")

In [87]:
for year in range(1985, 2021):
    train.train_rf(year, sierras_box, True)

2023-06-16 08:49:30,925 DEBUG: Load training data from a pickle file. [in train_rf at /home/jk871/fire-regen/src/processing/rf/train.py:29]
2023-06-16 08:49:33,456 DEBUG: Split data into training and testing. [in train_rf at /home/jk871/fire-regen/src/processing/rf/train.py:41]
2023-06-16 08:49:35,534 DEBUG: Prepare data for training. [in train_rf at /home/jk871/fire-regen/src/processing/rf/train.py:45]
2023-06-16 08:49:37,513 DEBUG: Start model training. [in train_rf at /home/jk871/fire-regen/src/processing/rf/train.py:57]
2023-06-16 08:50:24,481 DEBUG: Training complete. [in train_rf at /home/jk871/fire-regen/src/processing/rf/train.py:59]
2023-06-16 08:50:26,440 INFO: Year 1985 - Training rmse: 94.199723; R^2: 0.49265782528337476 [in train_rf at /home/jk871/fire-regen/src/processing/rf/train.py:66]
2023-06-16 08:50:26,442 INFO: Year 1985 - Validation error: 0.41521812667230196 [in train_rf at /home/jk871/fire-regen/src/processing/rf/train.py:67]
2023-06-16 08:50:26,442 INFO: Year 19